# CREATION AND UPLOADING EMBEDDINGS IN VECTOR DB


This section takes a cardiology-related PDF, breaks its pages into smaller chunks, encodes these chunks into numerical representations (embeddings) using Sentence Transformers, and then uploads these embeddings to a QDrant cloud instance for storage. This process allows for efficient storage and retrieval of semantic information from the document in the cloud-based database.

In [ ]:
!pip install transformers
!pip install sentence_transformers
!pip install qdrant-client
!pip -q install --upgrade together
!pip -q install langchain
!pip install fitz
!pip install PyMuPDF

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from jinja2 import Template
import pandas as pd
from qdrant_client.http import models
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client import QdrantClient
import os
import together
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from IPython.display import display, Markdown, Latex
import fitz
import grpc

In [ ]:
os.environ["TOGETHER_API_KEY"] = "148521c4088ad416dced465cc144671626b00c860af4e6ebc855953567087d8a"

In [ ]:
qdrant_client = QdrantClient(
    "https://a7f256c7-0d4f-4f06-a2be-7ea2de3e09fc.us-east4-0.gcp.cloud.qdrant.io:6333",
    prefer_grpc=True,
    api_key="7xkDRSDVb-jmBdGOFL5RuRCcV2UHPxCWfT3EbFrApwizz2EgZTClPA",
)

In [ ]:
def get_embeddings(text_batch):
  embedding_model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
  text_embeds = embedding_model.encode(text_batch)
  return text_embeds

In [ ]:
def create_QDrant_collection():
  embedding_model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
  qdrant_client.recreate_collection(
	collection_name="cardiology_docs",
	vectors_config=models.VectorParams(
		size=embedding_model.get_sentence_embedding_dimension(),
		distance=models.Distance.COSINE
	)
)

create_QDrant_collection()

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

model1 = SentenceTransformer('thenlper/gte-base')

In [ ]:
model1.get_sentence_embedding_dimension()

768

In [ ]:
def process_pdf_and_upload_to_qdrant(pdf_path):
    pdf_document = fitz.open(pdf_path)

    chunks = []
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        text = page.get_text("text")
        chunk_size = 5000
        chunks.extend([text[i:i + chunk_size] for i in range(0, len(text), chunk_size)])

    records_to_upload = []
    for idx, chunk in enumerate(chunks):
        vector = model1.encode(chunk).tolist()

        record = models.Record(
            id=idx,
            vector=vector,
            payload={"page_content": chunk}
        )
        records_to_upload.append(record)

    qdrant_client.upload_records(
        collection_name="cardiology_docs",
        records=records_to_upload
    )

    pdf_document.close()


In [ ]:
def upload_pdf():
  try:
      process_pdf_and_upload_to_qdrant("/content/cardiac_surgery.pdf")
  except grpc.RpcError as e:
      print(f"Error communicating with Qdrant: {e}")

upload_pdf()

In [ ]:
hits = qdrant_client.search(
	collection_name="cardiology_docs",
	query_vector=model1.encode("SPPB").tolist(),
	limit=3
)
for hit in hits:
	print(hit.payload, "score:", hit.score)

{'page_content': 'f 1 foot in front of and touching\nthe toes of the other foot for about 10 s)\n2. Gait speed (5-meters, see above), assessed 2 times\n3. Chair stand test (getting up from a chair with arms folded in\nfront of the chest 5 times)\nConsensus statements: The Short Physical Performance Battery as\na prediction tool for mortality after cardiac surgery\nUsage of the SPPB is advised to assess frailty in patients undergoing car-\ndiac surgery to predict intermediate-term mortality [45, 52].\nSPPB: Short Physical Performance Battery.\nConsensus statements: Composite indexes as prediction tools for\nmortality after cardiac surgery\nThe CAF test is advised to assess frailty in patients undergoing cardiac\nsurgery to predict short, intermediate- and long-term mortality [16, 21,\n54–56].\nThe EFT may be used to assess frailty in patients undergoing cardiac sur-\ngery to predict short and intermediate term mortality [48, 62].\nCAF: comprehensive assessment of frailty; EFT: Essential

# CHAT COMPLETION USING TOGETHER

Leveraging the Together.ai, this section uses the llama-2-70b-chat llm for chat completion. Moreover with the help of QDrant DB, the relavent docs are fetched and provided into the model as its internal knowledge.

In [ ]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, field_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = 'forbid'

    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text


In [ ]:
together.api_key = os.environ["TOGETHER_API_KEY"]
together.Models.start("togethercomputer/llama-2-7b-chat")

llm = TogetherLLM(
    model= "togethercomputer/llama-2-7b-chat",
    temperature = 0.1,
    max_tokens = 1024
)

embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1")

qdrant = Qdrant(
    client=qdrant_client,
    collection_name="cardiology_docs",
    embeddings=embeddings,
)

retriever = qdrant.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer in English:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, chain_type_kwargs=chain_type_kwargs)

# HeartGPT

HeartGPT, a chatbot, which takes any cardiology-related user query as input, retrieves the relavent docs from the vector db which is fed into the chat completion model to answer the users query.

In [ ]:
def print_result(result):
  output_text = f"""### Question:
  {query}
  ### Answer:
  {result}"""
  return (output_text)

In [ ]:
query = "Tell me about Short Physical Performance Battery"
result = qa.run(query)
display(Markdown(print_result(result)))

### Question:
  Tell me about Short Physical Performance Battery
  ### Answer:
  
The Short Physical Performance Battery (SPPB) is a predictive tool for mortality after cardiac surgery. It comprises 3 batteries of tests: balance (standing with feet side by side, semi-tandem, and tandem), gait speed, and the chair stand test. The SPPB has been shown to add predictive value for the estimation of perioperative and intermediate-term mortality. It is an easily accessible functional test with 3 components, and it has been used in 4 studies to predict 1-year mortality. The SPPB is a standardized subjective estimation of a patient's frailty status, but it is not an eyeball test and requires training.